# Image Classification with NeuroLite

This tutorial demonstrates how to build an image classification model using NeuroLite. We'll cover:

1. Data preparation and organization
2. Training a CNN model
3. Model evaluation and visualization
4. Making predictions on new images
5. Model deployment

## Dataset Structure

For image classification, NeuroLite expects your data to be organized in the following structure:

```
data/
├── train/
│   ├── class1/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   ├── class2/
│   │   ├── image1.jpg
│   │   └── ...
│   └── ...
└── test/ (optional)
    ├── class1/
    └── class2/
```

## Setup and Data Preparation

In [ ]:
import neurolite
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# For this tutorial, we'll create a simple synthetic dataset
# In practice, you would use your own image dataset

def create_sample_dataset():
    """Create a sample dataset for demonstration"""
    os.makedirs('sample_images/cats', exist_ok=True)
    os.makedirs('sample_images/dogs', exist_ok=True)
    
    # Create synthetic images (in practice, use real images)
    np.random.seed(42)
    
    # Generate "cat" images (more blue/gray)
    for i in range(20):
        img = np.random.randint(0, 100, (64, 64, 3), dtype=np.uint8)
        img[:, :, 2] += 100  # Add blue tint
        img = np.clip(img, 0, 255)
        Image.fromarray(img).save(f'sample_images/cats/cat_{i}.jpg')
    
    # Generate "dog" images (more red/brown)
    for i in range(20):
        img = np.random.randint(0, 100, (64, 64, 3), dtype=np.uint8)
        img[:, :, 0] += 100  # Add red tint
        img = np.clip(img, 0, 255)
        Image.fromarray(img).save(f'sample_images/dogs/dog_{i}.jpg')
    
    print("Sample dataset created!")
    print(f"Cats: {len(os.listdir('sample_images/cats'))} images")
    print(f"Dogs: {len(os.listdir('sample_images/dogs'))} images")

# Create the sample dataset
create_sample_dataset()

## Training an Image Classification Model

Now let's train a CNN model for image classification:

In [ ]:
# Train an image classification model
model = neurolite.train(
    data='sample_images/',
    model='resnet18',  # Use ResNet-18 architecture
    task='image_classification',
    image_size=64,  # Resize images to 64x64
    augmentation=True,  # Apply data augmentation
    validation_split=0.2,
    test_split=0.1,
    optimize=True  # Enable hyperparameter optimization
)

print("Model training completed!")
print(f"Model type: {type(model.model).__name__}")
print(f"Framework: {model.framework}")

## Model Evaluation

Let's examine the model's performance:

In [ ]:
# Print evaluation metrics
print("Model Evaluation Results:")
print("========================")

metrics = model.evaluation_results.metrics
for metric_name, value in metrics.items():
    if isinstance(value, float):
        print(f"{metric_name.capitalize()}: {value:.4f}")
    else:
        print(f"{metric_name.capitalize()}: {value}")

# Plot training history if available
if hasattr(model, 'training_history') and model.training_history:
    plt.figure(figsize=(12, 4))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    if 'loss' in model.training_history:
        plt.plot(model.training_history['loss'], label='Training Loss')
    if 'val_loss' in model.training_history:
        plt.plot(model.training_history['val_loss'], label='Validation Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    if 'accuracy' in model.training_history:
        plt.plot(model.training_history['accuracy'], label='Training Accuracy')
    if 'val_accuracy' in model.training_history:
        plt.plot(model.training_history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Show confusion matrix if available
if hasattr(model.evaluation_results, 'confusion_matrix') and model.evaluation_results.confusion_matrix is not None:
    import seaborn as sns
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        model.evaluation_results.confusion_matrix,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['cats', 'dogs'],
        yticklabels=['cats', 'dogs']
    )
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

## Making Predictions

Let's use our trained model to classify new images:

In [ ]:
# Create a test image for prediction
def create_test_image(image_type='cat'):
    """Create a test image similar to our training data"""
    np.random.seed(123)
    img = np.random.randint(0, 100, (64, 64, 3), dtype=np.uint8)
    
    if image_type == 'cat':
        img[:, :, 2] += 100  # Blue tint for cats
    else:
        img[:, :, 0] += 100  # Red tint for dogs
    
    img = np.clip(img, 0, 255)
    return img

# Create test images
test_cat = create_test_image('cat')
test_dog = create_test_image('dog')

# Save test images
Image.fromarray(test_cat).save('test_cat.jpg')
Image.fromarray(test_dog).save('test_dog.jpg')

# Display test images
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(test_cat)
plt.title('Test Cat Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(test_dog)
plt.title('Test Dog Image')
plt.axis('off')

plt.tight_layout()
plt.show()

# Make predictions
test_images = [test_cat, test_dog]
predictions = model.predict(test_images)

class_names = ['cats', 'dogs']
print("Predictions:")
for i, (img_type, pred) in enumerate(zip(['cat', 'dog'], predictions)):
    predicted_class = class_names[pred] if isinstance(pred, (int, np.integer)) else pred
    print(f"Test {img_type} image -> Predicted: {predicted_class}")

## Advanced Features

### Custom Image Size and Augmentation

In [ ]:
# Train with custom parameters
advanced_model = neurolite.train(
    data='sample_images/',
    model='efficientnet',  # Try a different architecture
    task='image_classification',
    image_size=128,  # Larger image size
    augmentation=True,
    augmentation_params={
        'rotation_range': 20,
        'width_shift_range': 0.1,
        'height_shift_range': 0.1,
        'horizontal_flip': True
    },
    validation_split=0.2,
    optimize=False  # Skip optimization for faster training
)

print("Advanced model training completed!")
print(f"Accuracy: {advanced_model.evaluation_results.metrics.get('accuracy', 'N/A'):.4f}")

### Model Comparison

In [ ]:
# Compare different models
models_to_compare = ['resnet18', 'resnet50', 'efficientnet']
results = {}

for model_name in models_to_compare:
    print(f"Training {model_name}...")
    try:
        trained_model = neurolite.train(
            data='sample_images/',
            model=model_name,
            task='image_classification',
            image_size=64,
            validation_split=0.2,
            optimize=False  # Skip optimization for speed
        )
        
        accuracy = trained_model.evaluation_results.metrics.get('accuracy', 0.0)
        results[model_name] = accuracy
        print(f"{model_name} accuracy: {accuracy:.4f}")
        
    except Exception as e:
        print(f"Failed to train {model_name}: {e}")
        results[model_name] = 0.0

# Plot comparison
if results:
    plt.figure(figsize=(10, 6))
    models = list(results.keys())
    accuracies = list(results.values())
    
    plt.bar(models, accuracies)
    plt.title('Model Comparison - Accuracy')
    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    
    # Add value labels on bars
    for i, v in enumerate(accuracies):
        plt.text(i, v + 0.01, f'{v:.3f}', ha='center')
    
    plt.tight_layout()
    plt.show()

## Model Deployment

Deploy your trained model for production use:

In [ ]:
# Export model to different formats
print("Deploying model...")

# Export to ONNX (cross-platform inference)
try:
    onnx_model = neurolite.deploy(model, format='onnx')
    print("✓ Model exported to ONNX format")
except Exception as e:
    print(f"✗ ONNX export failed: {e}")

# Export to TensorFlow Lite (mobile deployment)
try:
    tflite_model = neurolite.deploy(model, format='tflite')
    print("✓ Model exported to TensorFlow Lite format")
except Exception as e:
    print(f"✗ TFLite export failed: {e}")

# Create REST API (for demonstration, we won't actually start the server)
print("\nTo deploy as REST API:")
print("api_server = neurolite.deploy(model, format='api', port=8080)")
print("# This would start a server at http://localhost:8080")
print("# POST /predict with image data to get predictions")

## Best Practices for Image Classification

### 1. Data Quality
- Ensure images are properly labeled
- Use consistent image quality and resolution
- Balance your dataset across classes
- Remove corrupted or mislabeled images

### 2. Data Augmentation
- Use augmentation to increase dataset diversity
- Common augmentations: rotation, flipping, scaling, color jittering
- Be careful not to over-augment (can hurt performance)

### 3. Model Selection
- Start with ResNet-18 for quick experiments
- Use EfficientNet for better accuracy/efficiency trade-off
- Consider Vision Transformers (ViT) for large datasets

### 4. Training Tips
- Use transfer learning (pre-trained models)
- Monitor validation loss to avoid overfitting
- Use early stopping and learning rate scheduling
- Enable hyperparameter optimization for best results

### 5. Evaluation
- Look at per-class metrics, not just overall accuracy
- Examine confusion matrix for class-specific errors
- Test on real-world data similar to production

## Next Steps

- Try [Object Detection Tutorial](02_object_detection.ipynb) for detecting objects in images
- Explore [Custom Vision Models](03_custom_models.ipynb) to create your own architectures
- Learn about [Hyperparameter Optimization](../advanced/01_hyperparameter_optimization.ipynb)
- Check out [Model Deployment](../advanced/02_deployment.ipynb) for production deployment

## Summary

In this tutorial, you learned how to:

✓ Organize image data for NeuroLite
✓ Train CNN models with minimal code
✓ Evaluate model performance
✓ Make predictions on new images
✓ Compare different model architectures
✓ Deploy models for production use

NeuroLite makes computer vision accessible while maintaining the flexibility to customize when needed!